In [172]:
import pandas as pd
import json
import re
import codecs
with open('/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/finetune/raw/llm_finetune_db.finetune_pairs_v2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [173]:
for i, entry in enumerate(data):
    entry['id'] = int(i)
    # Xóa cột _id.$oid
    if '_id' in entry:
        del entry['_id']

In [174]:
# Convert the JSON data to a Pandas DataFrame
df = pd.json_normalize(data)

In [175]:
df.head()

,question,answer,references,request_link,id
0,Chi nhánh ngân hàng nước ngoài được quyết định...,Căn cứ theo khoản 1 Điều 17 Thông tư 11/2021/T...,[https://thuvienphapluat.vn/van-ban/Tien-te-Ng...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,0
1,"Đối với chi nhánh ngân hàng nước ngoài, việc x...",Căn cứ theo khoản 5 Điều 17 Thông tư 11/2021/T...,[https://thuvienphapluat.vn/van-ban/Tien-te-Ng...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,1
2,Hồ sơ xuất toán nợ ra khỏi ngoại bảng đối với ...,Căn cứ theo khoản 6 Điều 17 Thông tư 11/2021/T...,[https://thuvienphapluat.vn/van-ban/Tien-te-Ng...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,2
3,Trách nhiệm của cơ quan hải quan trong kiểm tr...,Theo quy định tại Điều 59 Luật Hải quan 2014 q...,[https://thuvienphapluat.vn/van-ban/Thuong-mai...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,3
4,Định mức thực tế sản xuất hàng hóa gia công...,Tại khoản 35 Điều 1 Thông tư 39/2018/TT-BTC sử...,[https://thuvienphapluat.vn/van-ban/Xuat-nhap-...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,4


In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114510 entries, 0 to 114509
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question      114510 non-null  object
 1   answer        114510 non-null  object
 2   references    114510 non-null  object
 3   request_link  114510 non-null  object
 4   id            114510 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 4.4+ MB


In [177]:
df.drop_duplicates(subset=['question'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110228 entries, 0 to 114508
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question      110228 non-null  object
 1   answer        110228 non-null  object
 2   references    110228 non-null  object
 3   request_link  110228 non-null  object
 4   id            110228 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 5.0+ MB


In [178]:
# Chuyển đổi kiểu dữ liệu cột
for col in df.columns:
    if col != 'id':
        df[col] = df[col].astype(str)

In [179]:
# Loại bỏ các bản ghi có giá trị null hoặc mảng rỗng ("[]") trong các cột quan trọng
df.replace(to_replace=["[]", "", None], value=pd.NA, inplace=True)
df.dropna(subset=['question', 'answer', 'references', 'request_link'], inplace=True)

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109846 entries, 0 to 114508
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question      109846 non-null  object
 1   answer        109846 non-null  object
 2   references    109846 non-null  object
 3   request_link  109846 non-null  object
 4   id            109846 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 5.0+ MB


In [181]:
df.head()

,question,answer,references,request_link,id
0,Chi nhánh ngân hàng nước ngoài được quyết định...,Căn cứ theo khoản 1 Điều 17 Thông tư 11/2021/T...,['https://thuvienphapluat.vn/van-ban/Tien-te-N...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,0
1,"Đối với chi nhánh ngân hàng nước ngoài, việc x...",Căn cứ theo khoản 5 Điều 17 Thông tư 11/2021/T...,['https://thuvienphapluat.vn/van-ban/Tien-te-N...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,1
2,Hồ sơ xuất toán nợ ra khỏi ngoại bảng đối với ...,Căn cứ theo khoản 6 Điều 17 Thông tư 11/2021/T...,['https://thuvienphapluat.vn/van-ban/Tien-te-N...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,2
3,Trách nhiệm của cơ quan hải quan trong kiểm tr...,Theo quy định tại Điều 59 Luật Hải quan 2014 q...,['https://thuvienphapluat.vn/van-ban/Thuong-ma...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,3
4,Định mức thực tế sản xuất hàng hóa gia công...,Tại khoản 35 Điều 1 Thông tư 39/2018/TT-BTC sử...,['https://thuvienphapluat.vn/van-ban/Xuat-nhap...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,4


In [182]:
link_counts = df['request_link'].value_counts()
frequency_df = link_counts.value_counts().rename_axis('count').reset_index(name='frequency')
print(frequency_df)

   count  frequency
0      3      30533
1      2       2677
2      4       2605
3      1        621
4      5        280
5      6         58
6      7          9
7      8          4
8      9          1


In [183]:
unique_counts = frequency_df['count'].unique()
count = 9
if count in unique_counts:
    print(f"\nCác liên kết xuất hiện {count} lần:")
    links = link_counts[link_counts == count].index.tolist()
    for link in links:
        print(link)
else:
        print(f"Không có liên kết nào xuất hiện {count} lần.")


Các liên kết xuất hiện 9 lần:
https://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/thong-tu-182022ttbldtbxh-bai-bo-11-thong-tu-ve-tien-luong-bao-hiem-xa-hoi-tu-ngay-15112022-41349.html


In [184]:
# Kiểm tra số lượng câu hỏi kết thúc bằng dấu chấm hỏi
num_with_question_mark = df['question'].str.endswith('?').sum()
print(f"Số lượng câu kết thúc bằng dấu chấm hỏi: {num_with_question_mark}")

Số lượng câu kết thúc bằng dấu chấm hỏi: 107379


In [185]:
# Lấy ra các câu không kết thúc bằng dấu chấm hỏi
dfnoq = df[~df['question'].str.endswith('?')]
dfnoq.head()

,question,answer,references,request_link,id
6,Quy định pháp luật về các từ viết tắt trong bả...,Căn cứ khoản 1 Điều 2 Quyết định 595/QĐ-BHXH n...,['https://thuvienphapluat.vn/van-ban/Bao-hiem/...,https://thuvienphapluat.vn/phap-luat/trong-tho...,6
8,Các trường hợp không được hưởng bảo hiểm y tế ...,Căn cứ Điều 23 Luật Bảo hiểm y tế 2008 quy địn...,['https://thuvienphapluat.vn/van-ban/Bao-hiem/...,https://thuvienphapluat.vn/phap-luat/trong-tho...,8
29,Thẩm quyền công bố mở bến cảng,Theo Điều 12 Nghị định 58/2017/NĐ-CP quy định ...,['https://thuvienphapluat.vn/van-ban/Giao-thon...,https://thuvienphapluat.vn/phap-luat/muon-dua-...,29
30,Thủ tục công bố mở bến cảng,Theo Điều 14 Nghị định 58/2017/NĐ-CP quy định ...,['https://thuvienphapluat.vn/van-ban/Giao-thon...,https://thuvienphapluat.vn/phap-luat/muon-dua-...,30
42,Nguyên tắc đăng ký tàu biển,Điều 18 Bộ luật Hàng hải Việt Nam 2015 quy địn...,['https://thuvienphapluat.vn/van-ban/Giao-thon...,https://thuvienphapluat.vn/phap-luat/trach-nhi...,42


In [186]:
df = df[df['question'].str.endswith('?')]

In [187]:
df.head(10)

,question,answer,references,request_link,id
0,Chi nhánh ngân hàng nước ngoài được quyết định...,Căn cứ theo khoản 1 Điều 17 Thông tư 11/2021/T...,['https://thuvienphapluat.vn/van-ban/Tien-te-N...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,0
1,"Đối với chi nhánh ngân hàng nước ngoài, việc x...",Căn cứ theo khoản 5 Điều 17 Thông tư 11/2021/T...,['https://thuvienphapluat.vn/van-ban/Tien-te-N...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,1
2,Hồ sơ xuất toán nợ ra khỏi ngoại bảng đối với ...,Căn cứ theo khoản 6 Điều 17 Thông tư 11/2021/T...,['https://thuvienphapluat.vn/van-ban/Tien-te-N...,https://thuvienphapluat.vn/phap-luat/doi-voi-c...,2
3,Trách nhiệm của cơ quan hải quan trong kiểm tr...,Theo quy định tại Điều 59 Luật Hải quan 2014 q...,['https://thuvienphapluat.vn/van-ban/Thuong-ma...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,3
4,Định mức thực tế sản xuất hàng hóa gia công...,Tại khoản 35 Điều 1 Thông tư 39/2018/TT-BTC sử...,['https://thuvienphapluat.vn/van-ban/Xuat-nhap...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,4
5,Khi doanh nghiệp tính định mức thực tế của sản...,Căn cứ tiểu mục 3 Mục 31 Bảng giải đáp thắc mắ...,['https://thuvienphapluat.vn/cong-van/Xuat-nha...,https://thuvienphapluat.vn/phap-luat/ho-tro-ph...,5
7,Bảo hiểm y tế được quy định như thế nào?,Căn cứ khoản 1 Điều 2 Luật Bảo hiểm y tế 2008 ...,['https://thuvienphapluat.vn/van-ban/Bao-hiem/...,https://thuvienphapluat.vn/phap-luat/trong-tho...,7
9,Trong thời gian nghỉ ốm đau công ty không đóng...,Căn cứ theo quy định tại khoản 5 Điều 42 Quyết...,['https://thuvienphapluat.vn/van-ban/Bao-hiem/...,https://thuvienphapluat.vn/phap-luat/trong-tho...,9
10,Doanh nghiệp kinh doanh casino có buộc phải xâ...,Căn cứ khoản 1 Điều 22 Nghị định 03/2017/NĐ-CP...,['https://thuvienphapluat.vn/van-ban/Thuong-ma...,https://thuvienphapluat.vn/phap-luat/doanh-ngh...,10
11,Quy chế giải quyết tranh chấp để xử lý tranh c...,Căn cứ khoản 1 Điều 22 Nghị định 03/2017/NĐ-CP...,['https://thuvienphapluat.vn/van-ban/Thuong-ma...,https://thuvienphapluat.vn/phap-luat/doanh-ngh...,11


In [188]:
# cleaned_file_path = 'finetuneda.json'
# df.to_json(cleaned_file_path, orient='records', force_ascii=False, indent=4)